In [1]:
!pip install google-generativeai python-docx python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\kotas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import docx
import os
from dotenv import load_dotenv
from google.api_core import exceptions # ★ エラーを特定するためにインポート

def get_text_from_docx(filepath):
    """docxファイルからテキストを抽出する"""
    doc = docx.Document(filepath)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


import google.generativeai as genai
import json
import time
import re

CHUNK_SIZE = 4500

# ▼▼▼▼▼ 新しいロジックで更新した最終版の関数 ▼▼▼▼▼
def create_embedding_with_fallback(text, title):
    """
    主要セクション → 全文 → チャンク化、という優先順位でEmbeddingを試行する。
    """
    # 1. 最初に処理対象とするテキストを決める
    print("  -> 主要セクションの抽出を試行...")
    text_to_process = extract_key_sections(text)
    source_description = "主要セクション"

    if not text_to_process:
        print("  -> 主要セクションが見つからず。全文を処理対象にします。")
        text_to_process = text
        source_description = "全文"
    else:
        print("  -> 主要セクションを抽出しました。")

    # 2. 決定したテキストでEmbedding生成を試行
    try:
        print(f"  -> 「{source_description}」でのEmbedding生成を試行...")
        response = genai.embed_content(model=model_for_embedding, content=text_to_process, task_type="RETRIEVAL_DOCUMENT", title=title)
        print(f"  -> 「{source_description}」での生成に成功。")
        return response['embedding']

    except exceptions.InvalidArgument as e:
        # サイズ上限エラーの場合のみ、最終手段のチャンク化へ進む
        if "payload size exceeds the limit" not in str(e):
            print(f"  -> 予期せぬエラー（サイズ上限以外）: {e}")
            return None
        
        # 3. 最終手段：チャンク化して再試行
        print(f"  -> 「{source_description}」が長すぎるため、最終手段「最初・真ん中・最後」で再試行します。")
        chunk_size = CHUNK_SIZE
        
        # テキストが短すぎて分割できない場合も考慮
        if len(text_to_process) < chunk_size * 2:
             chunked_text = text_to_process[:15000] # 安全のため上限より小さく
        else:
            start_text = text_to_process[:chunk_size]
            mid_point = len(text_to_process) // 2
            middle_text = text_to_process[mid_point - (chunk_size // 2) : mid_point + (chunk_size // 2)]
            end_text = text_to_process[-chunk_size:]
            chunked_text = f"【論文の冒頭】\n{start_text}\n\n【論文の中間部分】\n{middle_text}\n\n【論文の末尾】\n{end_text}"

        try:
            response = genai.embed_content(model=model_for_embedding, content=chunked_text, task_type="RETRIEVAL_DOCUMENT", title=title)
            print("  -> 最終手段（チャンク化）での生成に成功。")
            return response['embedding']
        except Exception as e_retry:
            print(f"  -> 最終手段での再試行中にエラーが発生: {e_retry}")
            return None
            
    except Exception as e:
        print(f"  -> 予期せぬエラーが発生しました: {e}")
        return None
    
# --- 実行部分 ---
papers_dir = './papers'
all_papers_data = []

print("論文の分析とEmbedding生成を開始します...")
docx_files = [f for f in os.listdir(papers_dir) if f.endswith('.docx')]
for docx_file in docx_files:
    filepath = os.path.join(papers_dir, docx_file)
    print(f"{docx_file}を処理中...")
    text = get_text_from_docx(filepath)
    if not text:
        continue

    # ★★★ 正しく、新しく定義した賢い関数を呼び出す ★★★
    embedding = create_embedding_with_fallback(text, docx_file)
    
    # embeddingが正常に生成された場合のみ、データをリストに追加
    if embedding:
        all_papers_data.append({
            "filename": docx_file,
            "content": text[:500] + "...", # プレビュー用に本文の冒頭を少しだけ保存
            "embedding": embedding
        })
    else:
        print(f"  -> {docx_file} のEmbedding生成に最終的に失敗しました。スキップします。")
    
    # APIレート制限のため少し待機
    time.sleep(2) 

# --- 最終結果をJSONファイルに書き出す ---
with open('papers.json', 'w', encoding='utf-8') as f:
    json.dump(all_papers_data, f, ensure_ascii=False, indent=2)

print("\n全ての論文のEmbedding生成が完了し、'papers.json'に保存しました。")

論文の分析とEmbedding生成を開始します...
02_内山田湖太_卒論_最終版.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションを抽出しました。
  -> 「主要セクション」でのEmbedding生成を試行...
  -> 「主要セクション」が長すぎるため、最終手段「最初・真ん中・最後」で再試行します。
  -> 最終手段（チャンク化）での生成に成功。
EI_16_田中慶人_卒論v4.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
kaku.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
kazuki.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
mae.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
naka.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
oisi - コピー - コピー.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全文を処理対象にします。
  -> 「全文」でのEmbedding生成を試行...
  -> 「全文」での生成に成功。
oisi - コピー.docxを処理中...
  -> 主要セクションの抽出を試行...
  -> 主要セクションが見つからず。全